In [11]:
# Connecting the fifa_world_cup database
import sqlite3
import pandas as pd

conn = sqlite3.Connection("fifa_world_cup.db")
cursor = conn.cursor()

In [13]:
# Add a new column to store the winning team (if it doesn't already exist)
try:
    cursor.execute("ALTER TABLE fifa_world_cup ADD COLUMN winning_team TEXT;")
except sqlite3.OperationalError:
    print("Column 'winning_team' already exists.")

# Update the new column with the team that had the higher score
cursor.execute("""
    UPDATE fifa_world_cup 
    SET winning_team = 
        CASE 
            WHEN home_team_score > away_team_score THEN home_team 
            WHEN away_team_score > home_team_score THEN away_team 
            ELSE 'Draw' 
        END;
""")

conn.commit()

# Query to find the longest winning streak
cursor.execute("""
    WITH Streaks AS (
        SELECT winning_team, date, 
            ROW_NUMBER() OVER (PARTITION BY winning_team ORDER BY date) - 
            ROW_NUMBER() OVER (ORDER BY date) AS streak_group
        FROM fifa_world_cup
        WHERE winning_team != 'Draw'
    ) 
    SELECT winning_team, COUNT(*) AS streak_length
    FROM Streaks
    GROUP BY winning_team, streak_group
    ORDER BY streak_length DESC
    LIMIT 1;
""")

longest_streak = cursor.fetchone()
if longest_streak:
    print(f"Team with the longest winning streak: {longest_streak[0]} ({longest_streak[1]} wins)")

# Close connection
conn.close()

Column 'winning_team' already exists.
Team with the longest winning streak: Argentina (3 wins)


In [ ]:
hta_df = pd.read_sql_query('''
    SELECT date, home_team, home_team_score
    FROM fifa_world_cup
    WHERE neutral_location = 0
    ORDER BY home_team_score DESC;
    ''', conn)